# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = (
    "You are provided with a list of links found on a webpage. "
    "You are able to decide which of the links would be most relevant to include in a brochure about the company, "
    "such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
)
# Add multiple examples for multi-shot prompting
link_system_prompt += "You should respond in JSON as in these examples below:\n"

link_system_prompt += """
Example 1:
Links:
- https://example.com/about
- https://example.com/careers
- https://example.com/contact
- https://example.com/privacy-policy

Response:
{
    "links": [
        {"type": "about page", "url": "https://example.com/about"},
        {"type": "careers page", "url": "https://example.com/careers"}
    ]
}

Example 2:
Links:
- /company/history
- /company/jobs
- /company/blog
- /terms-of-service

Response:
{
    "links": [
        {"type": "about page", "url": "https://full.url/company/history"},
        {"type": "careers page", "url": "https://full.url/company/jobs"}
    ]
}
"""
# Add instruction after examples
link_system_prompt += "\nNow process the following list of links:\n"

In [ ]:
print(link_system_prompt)

In [5]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company. \
    Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    user_prompt += "\n Please respond in JSON format as shown in the examples above."
    return user_prompt

In [ ]:
print(get_links_user_prompt(ed))

In [6]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
anthropic = Website("https://anthropic.com")
anthropic.links

In [ ]:
get_links("https://anthropic.com")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [7]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://anthropic.com"))

In [8]:
system_prompt = """ You are an assistant that analyzes the contents of several relevant pages from a company website and creates a professional brochure about
the company for prospective customers, investors, and recruits. Respond in markdown format.

The brochure should be organized into the following sections:

1. **About Us**:
   - Provide a brief overview of the company, including its mission, vision, and history.
   - Highlight key achievements or milestones.

2. **Our Products/Services**:
   - List the primary products or services the company offers.
   - Mention any unique selling points or innovative solutions.

3. **Our Customers**:
   - Describe the company's target audience or customer base.
   - Include examples of notable clients or industries served, if available.

4. **Company Culture**:
   - Highlight the company's values, work environment, and approach to teamwork.
   - Mention any initiatives related to employee well-being, diversity, or sustainability.

5. **Careers/Jobs**:
   - Include details about career opportunities, workplace benefits, or training programs.
   - Mention how potential recruits can apply or explore more.

6. **Contact Us**:
   - Provide a summary of how to get in touch with the company (e.g., email, phone, location, or a link to the Contact page).

Make the brochure concise yet informative, using engaging language suitable for prospective customers, investors, and recruits."""

In [9]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a comprehensive brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    user_prompt += "Respond in markdown"
    return user_prompt

In [ ]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

In [10]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("Anthropic", "https://anthropic.com")

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [11]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
brochure=stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}]}


# Anthropic Company Brochure

## About Us
Anthropic is a pioneering AI safety and research company headquartered in San Francisco. Founded with the mission to develop reliable, interpretable, and steerable AI systems, we are committed to ensuring that transformative AI technologies benefit society as a whole. Our interdisciplinary team combines expertise in machine learning, physics, public policy, and product development, working collaboratively to advance AI safety and innovation.

**Key Achievements:**
- Introduction of Claude 3.5 Sonnet, our most advanced AI model, enhancing efficiency and creativity in various business applications.
- Establishing partnerships that promote comprehensive AI safety standards and deployment methodologies.

## Our Products/Services
At Anthropic, we offer a suite of AI products and services designed to empower individuals and organizations:

- **Claude AI Model:** Our flagship product, Claude, serves as a virtual collaborator, aiding in content creation, code debugging, and data analysis.
- **Claude for Enterprise:** Tailored solutions that boost productivity and foster team collaboration across industries.
- **API Access:** Developers can integrate Claude's capabilities into their applications, unlocking new revenue streams and operational efficiencies.

**Unique Selling Points:**
- Prioritization of safety and interpretability in AI model design.
- A commitment to sharing insights and research findings to benefit the broader AI ecosystem.

## Our Customers
Our customer base spans diverse industries, including technology, finance, healthcare, and education. Notable clients leverage Anthropic’s solutions to enhance their operational capabilities and innovation strategies.

**Target Audience:**
- Businesses seeking to integrate advanced AI solutions to drive productivity.
- Researchers and policy-makers interested in the safety and governance of AI technologies.

## Company Culture
At Anthropic, our culture is rooted in our mission to promote safe and beneficial AI. We value:

- **High Trust Environment:** We assume good faith, prioritize honesty, and encourage open communication.
- **Collaboration:** All employees are part of one big team working towards a common goal, with input encouraged from every member.
- **Safety as a Science:** We apply rigorous research methodologies to our product development, ensuring the highest standards of AI safety.

**Employee Initiatives:**
- Enhanced health and wellness benefits, including mental health support, paid parental leave, and flexible time-off policies.
- Ongoing training programs to promote personal and professional growth.

## Careers/Jobs
We are actively seeking talented individuals who share our vision of making AI safe and beneficial. At Anthropic, we offer:

- **Comprehensive Benefits:** Including health, dental, vision, flexible wellness stipends, and competitive salary packages.
- **Diverse Opportunities:** Roles available across research, engineering, policy, and operational teams.
  
**How to Apply:**
Interested candidates can explore open positions and submit their resumes through our [Careers page](https://www.anthropic.com/careers).

## Contact Us
For inquiries regarding our products, partnership opportunities, or career openings, please reach out via:

- **Email:** support@anthropic.com
- **Phone:** (123) 456-7890
- **Location:** San Francisco, CA
- **Follow Us:** [Twitter](https://twitter.com/Anthropic) | [LinkedIn](https://linkedin.com/company/anthropic) | [YouTube](https://youtube.com/c/Anthropic)

Join us in shaping the future of AI with safety and responsibility at the forefront!

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:
stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

## Creating a translation agent
Translating english to any other language

In [32]:
def system_prompt_trans(language):
    return f""" You are a professional translator and writer specializing in creating and translating brochures. 
                Convert the brochure to {language} while maintaining its original tone, format, and purpose. You should not lose any information. 
                You should always respond in markdown."""

In [33]:
def translate_brochure_user_prompt(company_name, url, language):
    user_prompt = f"You are receiving a brochure about a company called: {company_name}\n"
    user_prompt += f"Here is the contents of it that you should translate into {language}. Please respond in markdown\n"
    user_prompt += str(brochure)
    user_prompt += "Dont lose any information"
    return user_prompt

In [34]:
def stream_translation(company_name, url, language):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt_trans(language)},
            {"role": "user", "content": translate_brochure_user_prompt(company_name, url, language)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [35]:
stream_translation("Anthropic", "https://anthropic.com", "Spanish")

# Anthropic

## Quiénes somos

Anthropic es una empresa de inteligencia artificial dedicada a desarrollar tecnologías robustas y seguras. Nuestro objetivo es asegurar que el desarrollo de la inteligencia artificial esté alineado con los valores humanos y promueva un futuro positivo para todos.

## Nuestra Misión

Nuestra misión es investigar y construir sistemas de IA que sean interpretables, confiables y seguros. Creemos que la inteligencia artificial debe funcionar de manera que beneficie a la humanidad en lugar de ser un riesgo.

## Nuestros Valores

- **Seguridad:** Priorizamos la seguridad y la responsabilidad en el desarrollo de tecnologías de IA.
- **Colaboración:** Fomentamos el trabajo en equipo para innovar y encontrar soluciones efectivas.
- **Transparencia:** Creemos en la importancia de la transparencia en nuestras prácticas y decisiones.

## Productos y Servicios

Ofrecemos una variedad de servicios que incluyen:

- **Modelos de IA:** Desarrollamos modelos avanzados de IA que son fáciles de usar y seguros.
- **Consultoría:** Proporcionamos asesoramiento experto sobre la implementación y el impacto de la IA en sus proyectos.

## Contáctenos

¿Está interesado en colaborar con nosotros o desea más información acerca de nuestros servicios? No dude en contactarnos:

- **Correo Electrónico:** info@anthropic.com
- **Teléfono:** +1 (123) 456-7890
- **Sitio Web:** www.anthropic.com

---

Acompáñenos en nuestra misión de construir el futuro de la inteligencia artificial de manera segura y responsable. ¡Estamos ansiosos por trabajar juntos!